# Setup

In [2]:
import json
import os
import os.path as osp

import pandas as pd
import numpy as np

In [3]:
if not os.getcwd().endswith('performance_dashboard'):
    os.chdir('..')

# Data Exploration

In [14]:
data_folder = osp.join('data', 'csv')


In [15]:
df = pd.read_csv(osp.join(data_folder,'2023_10_29_Full Match.csv'), sep=';', index_col=0)
df.index.name = 'Player'
df.head()

,Distanza,Distanza /min,Spesa Energetica,Dist > 15 km/h,Dist MP>20 W/Kg,%Dist > 15km/h,%Dist Acc>5 m/s2,%Dist Dec<-5 m/s2,%Dist MP>20 W/Kg,Potenza Met,...,T acc 2-3 m/s2,T acc 3-4 m/s2,T acc > 4 m/s2,T acc > 5 m/s2,RPE,T,Num Acc > 3 m/s2,Num Dec <-3 m/s2,SMax (kmh),Minutes
Player,,,,,,,,,,,,,,,,,,,,,
Team Average,9895,102,54,1897,2443,"19,2","0,1","0,2","24,6","9,2",...,00:01:55,00:00:37,00:00:18,00:00:05,"6,8",01:36:53,24,14,"29,4","96,89"
A,2279,122,13,469,612,"20,6","0,1","0,3","26,8","11,3",...,00:00:26,00:00:07,00:00:04,00:00:01,5,00:18:40,8,7,"31,3","18,68"
B,793,135,5,219,264,"27,6","0,2","0,4","33,2","12,8",...,00:00:09,00:00:04,00:00:01,00:00:00,"0,5",00:05:53,3,4,"28,1","5,89"
C,8893,98,48,1828,2297,"20,6","0,1","0,1","25,8","8,8",...,00:01:50,00:00:33,00:00:13,00:00:03,"8,5",01:30:23,24,10,"31,1","90,39"
D,6208,99,34,1302,1609,21,"0,2","0,1","25,9","8,9",...,00:01:14,00:00:22,00:00:14,00:00:05,5,01:02:57,12,3,"27,5","62,96"


In [16]:
df.iloc[0]

Distanza                  9895
Distanza /min              102
Spesa Energetica            54
Dist > 15 km/h            1897
Dist MP>20 W/Kg           2443
%Dist > 15km/h            19,2
%Dist Acc>5 m/s2           0,1
%Dist Dec<-5 m/s2          0,2
%Dist MP>20 W/Kg          24,6
Potenza Met                9,2
EDRel                      119
%Dist Equivalente         16,7
%Ind. Anaerobico          37,9
Dist 0-5 km/h             2352
Dist 5-10 km/h            2943
Dist 10-15 km/h           2703
Dist 15-20 km/h           1305
Dist 20-25 km/h            469
Dist > 25 km/h             123
T 0-5 km/h            00:51:48
T 5-10 km/h           00:25:36
T 10-15 km/h          00:13:18
T 15-20 km/h          00:04:36
T 20-25 km/h          00:01:16
T>15 km/h             00:06:09
T>25 km/h             00:00:16
D dec < -5 m/s2             17
D dec < -4 m/s2             53
D dec -4 & -3 m/s2          93
D dec -3 & -2 m/s2         260
D dec -2 & -1 m/s2         831
D acc 1-2 m/s2             973
D acc 2-

In [3]:
metrics_glossary = {
    'Distanza': {
        'metric_name': 'Distance',
        'description': 'Total distance covered during the match or session, measured in meters.',
        'lower_is_better': False
    },
    'Distanza /min': {
        'metric_name': 'Distance per minute',
        'description': 'Average distance covered per minute, measured in meters per minute.',
        'lower_is_better': False
    },
    'Spesa Energetica': {
        'metric_name': 'Energy Expenditure',
        'description': 'Total energy expenditure during the match or session, measured in kilocalories.',
        'lower_is_better': False
    },
    'Dist > 15 km/h': {
        'metric_name': 'Distance > 15 km/h',
        'description': 'Distance covered at speeds greater than 15 km/h, measured in meters.',
        'lower_is_better': False
    },
    'Dist MP>20 W/Kg': {
        'metric_name': 'Distance at MP > 20 W/Kg',
        'description': 'Distance covered while the metabolic power is greater than 20 W/kg, measured in meters.',
        'lower_is_better': False
    },
    '%Dist > 15km/h': {
        'metric_name': 'Percentage of Distance > 15 km/h',
        'description': 'Percentage of total distance covered at speeds greater than 15 km/h.',
        'lower_is_better': False
    },
    '%Dist Acc>5 m/s2': {
        'metric_name': 'Percentage of Distance with Acceleration > 5 m/s²',
        'description': 'Percentage of total distance covered with accelerations greater than 5 m/s².',
        'lower_is_better': False
    },
    '%Dist Dec<-5 m/s2': {
        'metric_name': 'Percentage of Distance with Deceleration < -5 m/s²',
        'description': 'Percentage of total distance covered with decelerations less than -5 m/s².',
        'lower_is_better': False
    },
    '%Dist MP>20 W/Kg': {
        'metric_name': 'Percentage of Distance at MP > 20 W/Kg',
        'description': 'Percentage of total distance covered while the metabolic power is greater than 20 W/kg.',
        'lower_is_better': False
    },
    'Potenza Met': {
        'metric_name': 'Metabolic Power',
        'description': 'Average metabolic power during the match or session, measured in W/kg.',
        'lower_is_better': False
    },
    'EDRel': {
        'metric_name': 'Equivalent Distance Relative',
        'description': 'Equivalent distance covered relative to the player’s physical capacity, measured in meters.',
        'lower_is_better': False
    },
    '%Dist Equivalente': {
        'metric_name': 'Percentage of Equivalent Distance',
        'description': 'Percentage of total distance covered relative to the player’s physical capacity.',
        'lower_is_better': False
    },
    '%Ind. Anaerobico': {
        'metric_name': 'Percentage of Anaerobic Index',
        'description': 'Percentage indicating the contribution of anaerobic activity during the match or session.',
        'lower_is_better': False
    },
    'Dist 0-5 km/h': {
        'metric_name': 'Distance 0-5 km/h',
        'description': 'Distance covered at speeds between 0 and 5 km/h, measured in meters.',
        'lower_is_better': False
    },
    'Dist 5-10 km/h': {
        'metric_name': 'Distance 5-10 km/h',
        'description': 'Distance covered at speeds between 5 and 10 km/h, measured in meters.',
        'lower_is_better': False
    },
    'Dist 10-15 km/h': {
        'metric_name': 'Distance 10-15 km/h',
        'description': 'Distance covered at speeds between 10 and 15 km/h, measured in meters.',
        'lower_is_better': False
    },
    'Dist 15-20 km/h': {
        'metric_name': 'Distance 15-20 km/h',
        'description': 'Distance covered at speeds between 15 and 20 km/h, measured in meters.',
        'lower_is_better': False
    },
    'Dist 20-25 km/h': {
        'metric_name': 'Distance 20-25 km/h',
        'description': 'Distance covered at speeds between 20 and 25 km/h, measured in meters.',
        'lower_is_better': False
    },
    'Dist > 25 km/h': {
        'metric_name': 'Distance > 25 km/h',
        'description': 'Distance covered at speeds greater than 25 km/h, measured in meters.',
        'lower_is_better': False
    },
    'T 0-5 km/h': {
        'metric_name': 'Time 0-5 km/h',
        'description': 'Time spent at speeds between 0 and 5 km/h, measured in minutes.',
        'lower_is_better': True
    },
    'T 5-10 km/h': {
        'metric_name': 'Time 5-10 km/h',
        'description': 'Time spent at speeds between 5 and 10 km/h, measured in minutes.',
        'lower_is_better': True
    },
    'T 10-15 km/h': {
        'metric_name': 'Time 10-15 km/h',
        'description': 'Time spent at speeds between 10 and 15 km/h, measured in minutes.',
        'lower_is_better': True
    },
    'T 15-20 km/h': {
        'metric_name': 'Time 15-20 km/h',
        'description': 'Time spent at speeds between 15 and 20 km/h, measured in minutes.',
        'lower_is_better': True
    },
    'T 20-25 km/h': {
        'metric_name': 'Time 20-25 km/h',
        'description': 'Time spent at speeds between 20 and 25 km/h, measured in minutes.',
        'lower_is_better': True
    },
    'T>15 km/h': {
        'metric_name': 'Time > 15 km/h',
        'description': 'Time spent at speeds greater than 15 km/h, measured in minutes.',
        'lower_is_better': False
    },
    'T>25 km/h': {
        'metric_name': 'Time > 25 km/h',
        'description': 'Time spent at speeds greater than 25 km/h, measured in minutes.',
        'lower_is_better': False
    },
    'D dec < -5 m/s2': {
        'metric_name': 'Distance with Deceleration < -5 m/s²',
        'description': 'Distance covered with decelerations less than -5 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D dec < -4 m/s2': {
        'metric_name': 'Distance with Deceleration < -4 m/s²',
        'description': 'Distance covered with decelerations less than -4 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D dec -4 & -3 m/s2': {
        'metric_name': 'Distance with Deceleration -4 & -3 m/s²',
        'description': 'Distance covered with decelerations between -4 and -3 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D dec -3 & -2 m/s2': {
        'metric_name': 'Distance with Deceleration -3 & -2 m/s²',
        'description': 'Distance covered with decelerations between -3 and -2 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D dec -2 & -1 m/s2': {
        'metric_name': 'Distance with Deceleration -2 & -1 m/s²',
        'description': 'Distance covered with decelerations between -2 and -1 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D acc 1-2 m/s2': {
        'metric_name': 'Distance with Acceleration 1-2 m/s²',
        'description': 'Distance covered with accelerations between 1 and 2 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D acc 2-3 m/s2': {
        'metric_name': 'Distance with Acceleration 2-3 m/s²',
        'description': 'Distance covered with accelerations between 2 and 3 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D acc 3-4 m/s2': {
        'metric_name': 'Distance with Acceleration 3-4 m/s²',
        'description': 'Distance covered with accelerations between 3 and 4 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D acc > 4 m/s2': {
        'metric_name': 'Distance with Acceleration > 4 m/s²',
        'description': 'Distance covered with accelerations greater than 4 m/s², measured in meters.',
        'lower_is_better': False
    },
    'D acc > 5 m/s2': {
        'metric_name': 'Distance with Acceleration > 5 m/s²',
        'description': 'Distance covered with accelerations greater than 5 m/s², measured in meters.',
        'lower_is_better': False
    },
    'T dec < -5 m/s2': {
        'metric_name': 'Time with Deceleration < -5 m/s²',
        'description': 'Time spent with decelerations less than -5 m/s², measured in minutes.',
        'lower_is_better': True
    },
    'T dec < -4 m/s2': {
        'metric_name': 'Time with Deceleration < -4 m/s²',
        'description': 'Time spent with decelerations less than -4 m/s², measured in minutes.',
        'lower_is_better': True
    },
    'T dec -4 & -3 m/s2': {
        'metric_name': 'Time with Deceleration -4 & -3 m/s²',
        'description': 'Time spent with decelerations between -4 and -3 m/s², measured in minutes.',
        'lower_is_better': True
    },
    'T dec -3 & -2 m/s2': {
        'metric_name': 'Time with Deceleration -3 & -2 m/s²',
        'description': 'Time spent with decelerations between -3 and -2 m/s², measured in minutes.',
        'lower_is_better': True
    },
    'T dec -2 & -1 m/s2': {
        'metric_name': 'Time with Deceleration -2 & -1 m/s²',
        'description': 'Time spent with decelerations between -2 and -1 m/s², measured in minutes.',
        'lower_is_better': True
    },
    'T acc 1-2 m/s2': {
        'metric_name': 'Time with Acceleration 1-2 m/s²',
        'description': 'Time spent with accelerations between 1 and 2 m/s², measured in minutes.',
        'lower_is_better': False
    },
    'T acc 2-3 m/s2': {
        'metric_name': 'Time with Acceleration 2-3 m/s²',
        'description': 'Time spent with accelerations between 2 and 3 m/s², measured in minutes.',
        'lower_is_better': False
    },
    'T acc 3-4 m/s2': {
        'metric_name': 'Time with Acceleration 3-4 m/s²',
        'description': 'Time spent with accelerations between 3 and 4 m/s², measured in minutes.',
        'lower_is_better': False
    },
    'T acc > 4 m/s2': {
        'metric_name': 'Time with Acceleration > 4 m/s²',
        'description': 'Time spent with accelerations greater than 4 m/s², measured in minutes.',
        'lower_is_better': False
    },
    'T acc > 5 m/s2': {
        'metric_name': 'Time with Acceleration > 5 m/s²',
        'description': 'Time spent with accelerations greater than 5 m/s², measured in minutes.',
        'lower_is_better': False
    },
    'RPE': {
        'metric_name': 'Rating of Perceived Exertion',
        'description': 'Self-reported measure of exercise intensity, typically on a scale from 6 to 20.',
        'lower_is_better': True
    },
    'T': {
        'metric_name': 'Total Time',
        'description': 'Total time of the match or session, measured in minutes.',
        'lower_is_better': True
    },
    'Num Acc > 3 m/s2': {
        'metric_name': 'Number of Accelerations > 3 m/s²',
        'description': 'Number of times accelerations greater than 3 m/s² occurred.',
        'lower_is_better': False
    },
    'Num Dec <-3 m/s2': {
        'metric_name': 'Number of Decelerations < -3 m/s²',
        'description': 'Number of times decelerations less than -3 m/s² occurred.',
        'lower_is_better': False
    },
    'SMax (kmh)': {
        'metric_name': 'Maximum Speed',
        'description': 'Maximum speed achieved during the match or session, measured in km/h.',
        'lower_is_better': False
    },
    'Minutes': {
        'metric_name': 'Minutes Played',
        'description': 'Total minutes played during the match or session.',
        'lower_is_better': False
    }
}

for m, m_dict in metrics_glossary.items():
    m_dict['name'] = m

In [4]:
metrics_glossary = [m_dict for m, m_dict in metrics_glossary.items()]

with open(osp.join('glossaries', 'metrics.json'), 'w') as f:
    json.dump(metrics_glossary, f) 

# Database

In [7]:
from sqlalchemy import create_engine
from utils.sql_queries import create_table, insert_table
from web_utils.data_manipulation import convert_to_seconds

In [5]:
db_path = osp.join('data', 'gps_data.db')
engine = create_engine(f'sqlite:///{db_path}', echo=False)

In [7]:
#File table
csv_folder = osp.join('data', 'csv')
data = []
for f in os.listdir(csv_folder):
    f_split = f.split('_')
    record = [
        '-'.join(f_split[:-1]), #date
        f_split[-1].split('.')[0] #type
    ]

    data.append(record)

df = pd.DataFrame(data, columns=['date', 'type'])
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.date

#Create the table
column_types = {
    'date':'Date',
    'type':'TEXT'
}

create_table(engine=engine, table_name='file_available', column_types=column_types, primary_keys=['date', 'type'])
insert_table(engine=engine, table_name='file_available', df=df, )

Inserting records: 100%|██████████| 1/1 [00:00<00:00, 137.60it/s]


In [21]:
print(data.shape)

(163, 55)


In [8]:
def convert_comma_float(value):
    """
    Convert a string representing a float with commas as decimal separators to a float.

    Args:
    - value (str): String representing a float with commas as decimal separators.

    Returns:
    - float: The converted float value.
    """
    if isinstance(value, str):
        # Replace the comma with a dot
        value = value.replace(',', '.')
        
    # Convert the string to a float
    return float(value)
    

In [11]:
#Metric table
csv_folder = osp.join('data', 'csv')
data = pd.DataFrame()
for f in os.listdir(csv_folder):
    f_split = f.split('_')
    temp_df = pd.read_csv(osp.join(csv_folder, f),  sep=';', index_col=0)
    temp_df = temp_df.reset_index(names='Player')
    temp_df['date'] = '-'.join(f_split[:-1])
    temp_df['type'] = f_split[-1].split('.')[0]
    data = pd.concat([data, temp_df])

for c in data.columns:
    if c.startswith('T'):
        data[c] = data[c].apply(convert_to_seconds)

#Create the table
column_types = {
    'Player': 'Text',
    'Distanza': 'REAL',
    'Distanza /min': 'REAL',
    'Spesa Energetica': 'REAL',
    'Dist > 15 km/h': 'REAL',
    'Dist MP>20 W/Kg': 'REAL',
    '%Dist > 15km/h': 'REAL',
    '%Dist Acc>5 m/s2': 'REAL',
    '%Dist Dec<-5 m/s2': 'REAL',
    '%Dist MP>20 W/Kg': 'REAL',
    'Potenza Met': 'REAL',
    'EDRel': 'REAL',
    '%Dist Equivalente': 'REAL',
    '%Ind. Anaerobico': 'REAL',
    'Dist 0-5 km/h': 'REAL',
    'Dist 5-10 km/h': 'REAL',
    'Dist 10-15 km/h': 'REAL',
    'Dist 15-20 km/h': 'REAL',
    'Dist 20-25 km/h': 'REAL',
    'Dist > 25 km/h': 'REAL',
    'T 0-5 km/h': 'REAL',
    'T 5-10 km/h': 'REAL',
    'T 10-15 km/h': 'REAL',
    'T 15-20 km/h': 'REAL',
    'T 20-25 km/h': 'REAL',
    'T>15 km/h': 'REAL',
    'T>25 km/h': 'REAL',
    'D dec < -5 m/s2': 'INTEGER',
    'D dec < -4 m/s2': 'INTEGER',
    'D dec -4 & -3 m/s2': 'INTEGER',
    'D dec -3 & -2 m/s2': 'INTEGER',
    'D dec -2 & -1 m/s2': 'INTEGER',
    'D acc 1-2 m/s2': 'INTEGER',
    'D acc 2-3 m/s2': 'INTEGER',
    'D acc 3-4 m/s2': 'INTEGER',
    'D acc > 4 m/s2': 'INTEGER',
    'D acc > 5 m/s2': 'INTEGER',
    'T dec < -5 m/s2': 'REAL',
    'T dec < -4 m/s2': 'REAL',
    'T dec -4 & -3 m/s2': 'REAL',
    'T dec -3 & -2 m/s2': 'REAL',
    'T dec -2 & -1 m/s2': 'REAL',
    'T acc 1-2 m/s2': 'REAL',
    'T acc 2-3 m/s2': 'REAL',
    'T acc 3-4 m/s2': 'REAL',
    'T acc > 4 m/s2': 'REAL',
    'T acc > 5 m/s2': 'REAL',
    'RPE': 'REAL',  
    'T': 'REAL',  # Total time is likely to be stored as REAL for precision
    'Num Acc > 3 m/s2': 'INTEGER',
    'Num Dec <-3 m/s2': 'INTEGER',
    'SMax (kmh)': 'REAL',
    'Minutes': 'REAL',
    'date':'TEXT',
    'type':'TEXT'
}

for c, t in column_types.items():
    if t == 'REAL':
        data[c] = data[c].apply(lambda x : convert_comma_float(x))

create_table(engine=engine, table_name='stats', column_types=column_types, primary_keys=['date', 'type', 'Player'])
insert_table(engine=engine, table_name='stats', df=data, )

print(data.shape)

Inserting records: 100%|██████████| 1/1 [00:00<00:00,  2.78it/s]

(163, 55)


In [10]:
data

,Player,Distanza,Distanza /min,Spesa Energetica,Dist > 15 km/h,Dist MP>20 W/Kg,%Dist > 15km/h,%Dist Acc>5 m/s2,%Dist Dec<-5 m/s2,%Dist MP>20 W/Kg,...,T acc > 4 m/s2,T acc > 5 m/s2,RPE,T,Num Acc > 3 m/s2,Num Dec <-3 m/s2,SMax (kmh),Minutes,date,type
0,Team Average,9895,102,54,1897,2443,"19,2","0,1","0,2","24,6",...,18,5,"6,8",5813,24,14,"29,4","96,89",2023-10-29,Full Match
1,A,2279,122,13,469,612,"20,6","0,1","0,3","26,8",...,4,1,5,1120,8,7,"31,3","18,68",2023-10-29,Full Match
2,B,793,135,5,219,264,"27,6","0,2","0,4","33,2",...,1,0,"0,5",353,3,4,"28,1","5,89",2023-10-29,Full Match
3,C,8893,98,48,1828,2297,"20,6","0,1","0,1","25,8",...,13,3,"8,5",5423,24,10,"31,1","90,39",2023-10-29,Full Match
4,D,6208,99,34,1302,1609,21,"0,2","0,1","25,9",...,14,5,5,3777,12,3,"27,5","62,96",2023-10-29,Full Match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,L,11784,121,62,2194,3130,"18,6",0,"0,1","26,6",...,9,2,9,5865,21,14,"27,9","97,76",2023-11-12,Full Match
11,M,8862,117,49,1334,2073,15,"0,1","0,3","23,4",...,12,3,"7,5",4556,21,23,"29,4","75,94",2023-11-12,Full Match
12,N,8819,116,48,1656,2242,"18,8","0,1","0,3","25,4",...,12,2,8,4550,27,21,"29,7","75,84",2023-11-12,Full Match
13,O,10639,110,58,1752,2570,"16,5","0,1","0,2","24,2",...,20,6,9,5828,24,17,"28,4","97,14",2023-11-12,Full Match


# End